In [70]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import gzip
import json
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [72]:
df = pd.read_csv('/content/drive/MyDrive/Syncable/sjsu/data-270/DATA 270 Project Files/dataset/combined.csv', low_memory=False)

In [73]:
df.duplicated().sum()

58255

In [74]:
df = df.drop_duplicates()
df.value_counts()

overall  verified  reviewTime   reviewerID      asin        reviewerName   reviewText                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [75]:
df1 = df.drop(['reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'image', 'vote', 'unixReviewTime'], axis=1)
df1

,overall,verified,reviewText,summary
0,5.0,True,It's good for beginners,Five Stars
1,5.0,True,I recommend this starter Ukulele kit. I has e...,Five Stars
2,5.0,True,G'daughter received this for Christmas present...,Learning new songs to play regularly
3,4.0,True,"According to my order history, I bought this t...",A bargain-bin good-enough ukulele that's held ...
4,1.0,True,Please pay attention better than I did to the ...,Poor Quality product.
...,...,...,...,...
749399,4.0,False,When I ordered this it was listed as Photo Edi...,File Management Software with Basic Editing Ca...
749400,3.0,False,This software has SO much going on. Theres a ...,"Might not be for the ""novice"""
749401,4.0,False,I have used both more complex and less complex...,"Great, Inexpensive Software for Those Who Have..."
749402,3.0,False,Pinnacle Studio 20 Ultimate is a perfectly ser...,Gets the job done ... but not as easy as it sh...


In [76]:
print(df1['reviewText'].dtype)


object


In [77]:
df1['verified'].value_counts()

verified
True     632902
False     58247
Name: count, dtype: int64

In [78]:
df2 = df1[df['verified'] == True]
df2

,overall,verified,reviewText,summary
0,5.0,True,It's good for beginners,Five Stars
1,5.0,True,I recommend this starter Ukulele kit. I has e...,Five Stars
2,5.0,True,G'daughter received this for Christmas present...,Learning new songs to play regularly
3,4.0,True,"According to my order history, I bought this t...",A bargain-bin good-enough ukulele that's held ...
4,1.0,True,Please pay attention better than I did to the ...,Poor Quality product.
...,...,...,...,...
749391,5.0,True,Arrived as described. I was pleased with purc...,I was pleased with
749392,1.0,True,Was not what I expected,Was not what I expected
749393,4.0,True,great buy,Four Stars
749394,4.0,True,"The product itself is quite good, in depth fix...",They put out a good product...all they need is...


In [79]:
df2 = df2[df2['overall'] >= 0]

In [80]:
mapping = {1: 0, 2: 0, 3: 1, 4: 2, 5: 2}
df3 = df2
df3['overall'] = df3['overall'].map(mapping)
df3

,overall,verified,reviewText,summary
0,2,True,It's good for beginners,Five Stars
1,2,True,I recommend this starter Ukulele kit. I has e...,Five Stars
2,2,True,G'daughter received this for Christmas present...,Learning new songs to play regularly
3,2,True,"According to my order history, I bought this t...",A bargain-bin good-enough ukulele that's held ...
4,0,True,Please pay attention better than I did to the ...,Poor Quality product.
...,...,...,...,...
749391,2,True,Arrived as described. I was pleased with purc...,I was pleased with
749392,0,True,Was not what I expected,Was not what I expected
749393,2,True,great buy,Four Stars
749394,2,True,"The product itself is quite good, in depth fix...",They put out a good product...all they need is...


In [81]:
df3 = df3.dropna(subset=['reviewText', 'summary'])
df3

,overall,verified,reviewText,summary
0,2,True,It's good for beginners,Five Stars
1,2,True,I recommend this starter Ukulele kit. I has e...,Five Stars
2,2,True,G'daughter received this for Christmas present...,Learning new songs to play regularly
3,2,True,"According to my order history, I bought this t...",A bargain-bin good-enough ukulele that's held ...
4,0,True,Please pay attention better than I did to the ...,Poor Quality product.
...,...,...,...,...
749391,2,True,Arrived as described. I was pleased with purc...,I was pleased with
749392,0,True,Was not what I expected,Was not what I expected
749393,2,True,great buy,Four Stars
749394,2,True,"The product itself is quite good, in depth fix...",They put out a good product...all they need is...


In [82]:
df3.loc[:, 'reviewTextSummary'] = df3['reviewText'] + " " + df3['summary']

<ipython-input-82-11592c74d0fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:, 'reviewTextSummary'] = df3['reviewText'] + " " + df3['summary']


In [83]:
df3 = df3.drop(['reviewText', 'summary'], axis=1)
df3

,overall,verified,reviewTextSummary
0,2,True,It's good for beginners Five Stars
1,2,True,I recommend this starter Ukulele kit. I has e...
2,2,True,G'daughter received this for Christmas present...
3,2,True,"According to my order history, I bought this t..."
4,0,True,Please pay attention better than I did to the ...
...,...,...,...
749391,2,True,Arrived as described. I was pleased with purc...
749392,0,True,Was not what I expected Was not what I expected
749393,2,True,great buy Four Stars
749394,2,True,"The product itself is quite good, in depth fix..."


In [84]:
df3['overall'].value_counts()

overall
2    558924
1     38471
0     35014
Name: count, dtype: int64

In [85]:
missing_values = df3['reviewTextSummary'].isnull().sum()
missing_values

0

In [86]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [87]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [88]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

unzip:  cannot find or open /usr/share/nltk_data/corpora/wordnet.zip, /usr/share/nltk_data/corpora/wordnet.zip.zip or /usr/share/nltk_data/corpora/wordnet.zip.ZIP.


In [89]:
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
stop_word_group = stopwords.words('english')

lematizer = WordNetLemmatizer()

def preprocess_text(textblob):
    textblob = re.sub(r'\d', '', textblob, re.UNICODE)
    textblob = re.sub(r'[^\w\s]', '', textblob, re.UNICODE)
    textblob = textblob.lower()
    textblob = [lematizer.lemmatize(token) for token in textblob.split(" ")]
    textblob = [word for word in textblob if not word in stop_word_group] # removing any stopwords
    textblob = " ".join(textblob)
    return textblob

# def preprocess_text(textblob):
#     try:
#         textblob = re.sub(r'\d', '', textblob, re.UNICODE)
#         textblob = re.sub(r'[^\w\s]', '', textblob, re.UNICODE)
#         textblob = textblob.lower()
#         textblob = [lematizer.lemmatize(token) for token in textblob.split(" ")]
#         textblob = [word for word in textblob if not word in stop_word_group] # removing any stopwords
#         textblob = " ".join(textblob)
#     except:
#         print(textblob)
#         return textblob
#     return textblob


df3['reviewTextSummary'] = df3.reviewTextSummary.apply(lambda x: preprocess_text(x))
df3

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,overall,verified,reviewTextSummary
0,2,True,good beginner five star
1,2,True,recommend starter ukulele kit ha everything y...
2,2,True,gdaughter received christmas present last year...
3,2,True,according order history bought kit towards end...
4,0,True,please pay attention better review poor qualit...
...,...,...,...
749391,2,True,arrived described wa pleased purchase wa pleased
749392,0,True,wa expected wa expected
749393,2,True,great buy four star
749394,2,True,product quite good depth fix easy auto feature...


In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
# vvecTool = TfidfVectorizer(max_features=2000)
# data_features = vvecTool.fit_transform(df3['reviewTextSummary'])
# data_features = data_features.toarray()
# print(data_features)

In [92]:
y = df3['overall']

In [93]:
from sklearn.model_selection import train_test_split

#X_train, X_tmp, y_train, y_tmp = train_test_split(data_features, y, test_size=0.2, random_state=42)
X_train, X_tmp, y_train, y_tmp = train_test_split(df3['reviewTextSummary'], y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=42)



In [94]:
vvecTool = TfidfVectorizer()
vvecTool.fit(df3['reviewTextSummary'])
X_train = vvecTool.fit_transform(X_train)
X_val = vvecTool.transform(X_val)
X_test = vvecTool.transform(X_test)

# Tuned Parameters using GridSearchCV

In [98]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from scipy.stats import randint
# from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


#mod = XGBClassifier(objective='multi:softmax', num_class=3, tree_method='gpu_hist')

mod = XGBClassifier(objective='multi:softmax', num_class=3, tree_method='hist', device='cuda', eval_metric='mlogloss')
param_grid = {
    'max_depth': [5, 10, 15],
    'n_estimators': [100, 200, 400],
    'learning_rate': [0.01, 0.1],
    'gamma': [0.1, 0.5],
}
r_search = GridSearchCV(estimator=mod, param_grid=param_grid, cv=3, verbose=3)

r_search.fit(X_val, y_val)

best_params = r_search.best_params_
print("Best parameters:", best_params)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV 1/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100;, score=0.901 total time=   2.1s
[CV 2/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100;, score=0.901 total time=   2.1s
[CV 3/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=100;, score=0.901 total time=   2.1s
[CV 1/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=200;, score=0.901 total time=   3.8s
[CV 2/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=200;, score=0.901 total time=   4.0s
[CV 3/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=200;, score=0.901 total time=   3.8s
[CV 1/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=400;, score=0.901 total time=   7.2s
[CV 2/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=400;, score=0.901 total time=   7.6s
[CV 3/3] END gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=400;, score=

# Old Code Used for testing Purposes

In [99]:
# from sklearn.utils.class_weight import compute_sample_weight
# sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)
# sample_weights

array([0.37725362, 0.37725362, 0.37725362, ..., 0.37725362, 0.37725362,
       0.37725362])

In [100]:
# base_model = XGBClassifier(objective='multi:softmax',n_class=3, random_state=42, tree_method='hist', device='cuda')
# base_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:25:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_class" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_class=3, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [101]:
# y_base_pred = base_model.predict(X_test)
# y_base_pred_roc = base_model.predict_proba(X_test)

In [102]:
# print("Accuracy:", accuracy_score(y_test, y_base_pred))
# print("Precision:", precision_score(y_test, y_base_pred, average='macro'))
# print("Recall:", recall_score(y_test, y_base_pred, average='macro'))
# print("AUC: ", roc_auc_score(y_test, y_base_pred_roc, average = 'macro', multi_class='ovr'))

Accuracy: 0.9002118889743941
Precision: 0.8386215974546399
Recall: 0.44749247639564244
AUC:  0.5897083262999403


In [103]:
# from sklearn.metrics import roc_curve
# import xgboost as xgb
# from xgboost import XGBClassifier

# model = XGBClassifier(**best_params, objective='multi:softmax', n_class=3, random_state=42, tree_method='hist', device='cuda')
# #model = XGBClassifier(n_estimators = 800, max_depth = None, learning_rate = 0.01, gamma = 0.1, objective='multi:softmax', n_class=3, random_state=42, tree_method='hist', device='cuda')
# #, ojective='multi:softprob', eval_metric='auc'
# #model = XGBClassifier(n_estimators = 100, max_depth = 10, learning_rate = 0.1, gamma = 0.5, objective='multi:softmax', n_class=3, random_state=42, tree_method='hist', device='cuda')


# #model.fit(X_train, y_train, sample_weight=sample_weights)
# model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:25:33] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_class" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_class=3, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [104]:
# y_pred = model.predict(X_test)
# y_pred_roc = model.predict_proba(X_test)

In [105]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import precision_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import roc_auc_score

# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Precision:", precision_score(y_test, y_pred, average='macro'))
# print("Recall:", recall_score(y_test, y_pred, average='macro'))
# print("AUC: ", roc_auc_score(y_test, y_pred_roc, average = 'macro', multi_class='ovr'))

Accuracy: 0.9007389759753744
Precision: 0.921273364749572
Recall: 0.4349005574783509
AUC:  0.5915647725119869


In [106]:
# from sklearn.metrics import classification_report

# report = classification_report(y_test, y_base_pred)
# print(report)


              precision    recall  f1-score   support

           0       0.67      0.12      0.21      5348
           1       0.94      0.22      0.36      5714
           2       0.90      1.00      0.95     83799

    accuracy                           0.90     94861
   macro avg       0.84      0.45      0.51     94861
weighted avg       0.89      0.90      0.87     94861



In [107]:
# from sklearn.metrics import classification_report

# report = classification_report(y_test, y_pred)
# print(report)


              precision    recall  f1-score   support

           0       0.88      0.08      0.15      5348
           1       0.99      0.22      0.37      5714
           2       0.90      1.00      0.95     83799

    accuracy                           0.90     94861
   macro avg       0.92      0.43      0.49     94861
weighted avg       0.90      0.90      0.87     94861

